In [4]:
import yfinance as yf
import talib
import twstock
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf_device='/gpu:0'

In [20]:
def get_code():
    stock_codes = twstock.twse
    return list(stock_codes)

def get_data(code):
    stock = yf.Ticker(code + '.TW')
    history = stock.history('60d', interval='5m')
    return history

In [22]:
stock_codes = get_code()
stock_data = get_data(stock_codes[0])
data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]

In [25]:
data['Volume'] = data['Volume'].astype(np.float32)/1000
data

/tmp/ipykernel_1905/1324377846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Volume'] = data['Volume'].astype(np.float32)/1000


,Open,High,Low,Close,Volume
Datetime,,,,,
2024-02-20 09:00:00+08:00,32.799999,32.900002,32.549999,32.599998,0.000000
2024-02-20 09:05:00+08:00,32.599998,32.650002,32.549999,32.599998,242.000000
2024-02-20 09:10:00+08:00,32.599998,32.650002,32.549999,32.599998,223.449997
2024-02-20 09:15:00+08:00,32.599998,32.650002,32.549999,32.599998,126.297997
2024-02-20 09:20:00+08:00,32.599998,32.650002,32.599998,32.599998,91.791000
...,...,...,...,...,...
2024-05-17 13:00:00+08:00,33.900002,33.950001,33.849998,33.900002,572.202026
2024-05-17 13:05:00+08:00,33.900002,33.950001,33.900002,33.900002,319.604004
2024-05-17 13:10:00+08:00,33.900002,33.950001,33.900002,33.950001,265.378998


In [9]:
def attention(query, keys, values):
    # 计算注意力权重
    scores = tf.matmul(query, keys, transpose_b=True) / tf.math.sqrt(tf.cast(keys.shape[-1], tf.float32))
    attention_weights = tf.nn.softmax(scores, axis=-1)
    # 加权平均值
    context = tf.matmul(attention_weights, values)
    return context, attention_weights

class LSTMWithAttention(tf.keras.layers.Layer):
    def __init__(self, units, attention_dim):
        super(LSTMWithAttention, self).__init__()
        self.units = units
        self.attention_dim = attention_dim
        self.lstm_cell = tf.keras.layers.LSTMCell(units)
        self.W_q = tf.keras.layers.Dense(attention_dim)
        self.W_k = tf.keras.layers.Dense(attention_dim)
        self.W_v = tf.keras.layers.Dense(attention_dim)
        self.W_hq = tf.keras.layers.Dense(units)
    
    def call(self, inputs, states):
        outputs = []
        attention_weights_all = []
        H, C = states
        for t in range(inputs.shape[1]):
            X = inputs[:, t, :]
            # LSTM的时间步操作
            _, (H, C) = self.lstm_cell(X, states=[H, C])
            # 计算注意力上下文向量
            query = self.W_q(H)
            keys = self.W_k(inputs)
            values = self.W_v(inputs)
            context, attention_weights = attention(tf.expand_dims(query, axis=1), keys, values)
            context = tf.squeeze(context, axis=1)
            # 结合LSTM输出和注意力上下文向量
            H_combined = tf.concat([H, context], axis=-1)
            Y = self.W_hq(H_combined)
            outputs.append(Y)
            attention_weights_all.append(attention_weights)
        outputs = tf.stack(outputs, axis=1)
        return outputs, (H, C), tf.stack(attention_weights_all, axis=1)


In [11]:
class Seq2SeqModel(tf.keras.Model):
    def __init__(self, units, attention_dim, output_dim):
        super(Seq2SeqModel, self).__init__()
        self.lstm_with_attention = LSTMWithAttention(units, attention_dim)
        self.dense = tf.keras.layers.Dense(output_dim)
    
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        initial_state = [
            tf.zeros((batch_size, self.lstm_with_attention.units)),
            tf.zeros((batch_size, self.lstm_with_attention.units))
        ]
        lstm_outputs, _, attention_weights = self.lstm_with_attention(inputs, initial_state)
        outputs = self.dense(lstm_outputs)
        return outputs, attention_weights

# 定义超参数
units = 64
attention_dim = 32
output_dim = 10

# 构建模型
model = Seq2SeqModel(units, attention_dim, output_dim)

# 编译模型
model.compile(optimizer='adam', loss='mse')

# 生成一些随机数据进行测试
inputs = tf.random.normal([32, 10, 8])  # (batch_size, sequence_length, feature_dim)
outputs, attention_weights = model(inputs)
print(outputs.shape)  # 预期输出形状: (32, 10, output_dim)
print(attention_weights.shape)  # 预期输出形状: (32, 10, 10)
model.summary()

(32, 10, 10)
(32, 10, 1, 10)
Model: "seq2_seq_model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_with_attention_2 (LST  multiple                  27552     
 MWithAttention)                                                 
                                                                 
 dense_14 (Dense)            multiple                  650       
                                                                 
Total params: 28202 (110.16 KB)
Trainable params: 28202 (110.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 假设我们有一些训练数据
X_train = tf.random.normal([1000, 10, 8])
y_train = tf.random.normal([1000, 10, output_dim])

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=32)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node mean_squared_error_1/SquaredDifference defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 17, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 529, in dispatch_queue

  File "/usr/local/lib/python3.